<a href="https://colab.research.google.com/github/kazuki-komori/coursera/blob/main/regression/CSR_regression_w4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Coursera 回帰 第4回

https://www.coursera.org/learn/ml-regression/supplement/MhFOa/polynomial-regression

このノートでは、さまざまな回帰モデルを比較して、どのモデルが最も適合しているかを評価します。このトピックを検討する手段として、多項式回帰を使用します。特に、以下のことを行います。

- 配列と次数を受け取り、各列が次数の合計までの多項式値の配列となるデータフレームを返す関数を作成せよ。

  Write a function to take an an array and a degree and return an data frame where each column is the array to a polynomial value up to the total degree.

- プロットツール（例：matplotlib）を使って、多項式回帰を可視化する。

  Use a plotting tool (e.g. matplotlib) to visualize polynomial regressions

- プロットツール(例：matplotlib)を使って、データの異なるサブセットで同じ多項式次数を可視化する

  Use a plotting tool (e.g. matplotlib) to visualize the same polynomial degree on different subsets of the data

- 検証セットを使って、多項式次数を選択する

  Use a validation set to select a polynomial degree

- テストデータを使って最終的な適合度を評価する

  Assess the final fit using test data


## Setup

In [ ]:
# unzip datasets
!unzip -q /content/drive/MyDrive/coursera/home_data.zip
!unzip -q /content/drive/MyDrive/coursera/kc_house_data.csv.zip
# install libraries
!pip install turicreate


     |████████████████████████████████| 92.0 MB 14 kB/s 
     |████████████████████████████████| 86.4 MB 68 kB/s 
     |████████████████████████████████| 322 kB 40.5 MB/s 
     |████████████████████████████████| 3.6 MB 45.4 MB/s 
     |████████████████████████████████| 3.5 MB 42.6 MB/s 
     |████████████████████████████████| 18.3 MB 432 kB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 3.8 MB 41.4 MB/s 
     |████████████████████████████████| 2.9 MB 47.6 MB/s 
     |████████████████████████████████| 449 kB 45.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=a1e9a3963f72f87bccd646d1ebab3b3fba05508df08b2c2091986cce85f08d1d
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320859 sha256=ce2f5dbe2d8

In [ ]:
import turicreate
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df_kc_house = pd.read_csv('/content/kc_house_data.csv')
df_kc_house.head(5)

## (2) 指定された累乗を返す関数を作成する

In [ ]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    df_out = pd.DataFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    df_out["power_1"] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        for power in range(2, degree+1):
            # first we'll give the column a name:
            # assign poly_sframe[name] to be feature^power
            df_out[f"power_{power}"] = feature.apply(lambda row: row**power)
    return df_out

## (3) `sqft_living`, `price` でデータをソートする

In [ ]:
df_kc_house = df_kc_house.sort_values(by=['sqft_living', 'price'])
df_kc_house[['sqft_living', 'price']].head(5)

## (4) (5) 1次の項を作ってみる

In [ ]:
df_poly1 = polynomial_sframe(df_kc_house["sqft_living"], 1)
df_poly1["price"] = df_kc_house["price"]

## (6) ライブラリを使って1次式に対して単回帰を実装してみる

In [ ]:
model1 = turicreate.linear_regression.create(turicreate.SFrame(df_poly1), target = 'price', features = ['power_1'], validation_set = None)

## (7) 当てはまりを可視化してみる

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(df_poly1['power_1'], df_poly1['price'], '.', df_poly1['power_1'], model1.predict(turicreate.SFrame(df_poly1)), '-')
plt.show()

## (8) 次数を上げて plot してみる

In [ ]:
def plot_polynominal(start_n = 1, end_n = 5, df=df_kc_house):
  for n in range(start_n, end_n+1):
    # 特徴量を作成
    df_poly = polynomial_sframe(df["sqft_living"], n)
    df_poly["price"] = df["price"]
    _feature_col = df_poly.columns.drop(["price"]).to_list()
    # fit
    model = turicreate.linear_regression.create(turicreate.SFrame(df_poly), target = 'price', features = _feature_col, validation_set = None, verbose=False)
    # 予測
    _pred = model.predict(turicreate.SFrame(df_poly))
    # plot
    print(f"{n}次式")
    plt.rcParams["figure.figsize"] = (10, 4)
    plt.plot(df_poly[_feature_col], df_poly['price'], '.', df_poly[_feature_col], _pred, '-')
    plt.show()

plot_polynominal()

## (9) 15次式plot

明らかに過学習w

In [ ]:
plot_polynominal(start_n = 15, end_n = 15)

## (10) データの分割

In [ ]:
_df1, _df2 = turicreate.SFrame(df_kc_house).random_split(.5, seed=0)
df_set1, df_set2 = turicreate.SFrame(_df1).random_split(.5, seed=0)
df_set3, df_set4 = turicreate.SFrame(_df2).random_split(.5, seed=0)

## (11) 4つの set を 次数15のモデルに適用する

In [ ]:
plot_polynominal(start_n = 15, end_n = 15, df = df_set1)
plot_polynominal(start_n = 15, end_n = 15, df = df_set2)
plot_polynominal(start_n = 15, end_n = 15, df = df_set3)
plot_polynominal(start_n = 15, end_n = 15, df = df_set4)

## (14) テストデータの分割

In [ ]:
df_training_and_validation, df_test = turicreate.SFrame(df_kc_house).random_split(.9, seed=1)
df_train, df_valid = df_training_and_validation.random_split(.5, seed=1)

## (15) 次数1-15の間で検証する

In [ ]:
def polynominal_with_valid(start_n = 1, end_n = 5):
  rss_arr = []
  for n in range(start_n, end_n+1):
    # 特徴量を作成
    df_poly = polynomial_sframe(df_train["sqft_living"], n)
    df_poly["price"] = df_train["price"]
    _feature_col = df_poly.columns.drop(["price"]).to_list()
    # fit
    model = turicreate.linear_regression.create(turicreate.SFrame(df_poly), target = 'price', features = _feature_col, validation_set = None, verbose=False)
    # 予測
    df_poly_valid = polynomial_sframe(df_valid["sqft_living"], n)
    _pred = model.predict(turicreate.SFrame(df_poly_valid))
    # rss
    _rss = ((df_valid["price"].to_numpy() - _pred)**2).sum()
    print(f"次数: {n}\t RSS: {_rss: .2f}\n")
    rss_arr.append(_rss)
  # 可視化
  fig, ax = plt.subplots(figsize=(16, 4))
  ax.set_ylim((5.5e14, 7e14))
  sns.barplot(x=list(range(1, 1+end_n)), y=rss_arr)
  plt.show()
  return rss_arr

rss = polynominal_with_valid(end_n=15)

## (16) 最も RSS が小さいのは？
N = 7

## (17) N = 7 で TEST の RSS を計算する

In [ ]:
def calc_rss_test(N=1):
  # 特徴量を作成
  df_poly = polynomial_sframe(df_train["sqft_living"], N)
  df_poly["price"] = df_train["price"]
  _feature_col = df_poly.columns.drop(["price"]).to_list()
  # fit
  model = turicreate.linear_regression.create(turicreate.SFrame(df_poly), target = 'price', features = _feature_col, validation_set = None, verbose=False)
  # 予測
  df_poly_test = polynomial_sframe(df_test["sqft_living"], N)
  _pred = model.predict(turicreate.SFrame(df_poly_test))
  # rss
  _rss = ((df_test["price"].to_numpy() - _pred)**2).sum()
  print(f"次数: {N}\t RSS: {_rss: .2f}\n")
  return _rss

calc_rss_test(7)

In [ ]:
rss_arr = []
for n in range(1, 16):
  rss_arr.append(calc_rss_test(n))

# 可視化
fig, ax = plt.subplots(figsize=(16, 4))
ax.set_ylim((1.2e14, 1.3e14))
sns.barplot(x=list(range(1, 1+15)), y=rss_arr)
plt.show()